## Import cities_df

In [30]:
RERUN_TIME_IN_HOURS = 24
WEATHER_FORCAST_TIMEWINDOW = 3

In [31]:
import pandas as pd
import sqlalchemy
sys.path.append('..\keys')
import key

schema="gans_db"   # name of the database you want to use here
host="localhost"        # to connect to your local server
user="root"
password=key.SQL_PASSWORD # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
cities_df = pd.read_sql_table('city',con=con)

In [33]:
cities_df

,city_id,city,country,population,lat,lon
0,1,Istanbul,Turkey,15840900,41.0136,28.95500
1,2,Moscow,Russia,12632409,55.7500,37.61670
2,3,London,United Kingdom,8799800,51.5072,-0.12750
3,4,Saint Petersburg,Russia,5376672,59.9500,30.30000
4,5,Berlin,Germany,3677472,52.5167,13.38330
5,6,Madrid,Spain,3305408,40.3833,-3.71667
6,7,Kyiv,Ukraine,2962180,50.4500,30.52330
7,8,Rome,Italy,2761632,41.9000,12.50000
8,9,Baku,Azerbaijan,2303100,40.4000,49.90000
9,10,Bucharest,Romania,2161347,44.4325,26.10390


## Get Weather from API

In [34]:
import requests
import json
import math

#Hence 
cities_weath = {"city_id":[]
                ,"forcast_time":[]
                ,"outlook":[]
                ,"temperature":[]
                ,"feels_like":[]
                ,"wind_speed":[]
                ,"rain_prob":[]
                }
for i in range(len(cities_df)):
    lat=cities_df.lat[i]
    lon=cities_df.lon[i]
    url_weather= f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={key.Weather_API_key}&units=metric"
    weather_response = requests.get(url_weather)
    if weather_response.status_code != 200:
        print(weather_response.text)
    else:
        city_json = weather_response.json()
        #API interface provide weather forcast for the next 5 days(per 3 hours)
        iterate = math.ceil(RERUN_TIME_IN_HOURS/WEATHER_FORCAST_TIMEWINDOW)
        for forcast in city_json["list"][:iterate+1]:
            cities_weath["city_id"].append(cities_df.city_id[i])
            cities_weath["forcast_time"].append(forcast["dt_txt"])
            cities_weath["temperature"].append(forcast["main"]["temp"])
            cities_weath["outlook"].append(forcast["weather"][0]["description"])
            cities_weath["feels_like"].append(forcast["main"]["feels_like"])
            cities_weath["wind_speed"].append(forcast["wind"]["speed"])
            cities_weath["rain_prob"].append(forcast["pop"])
       

In [35]:
cities_weather_df = pd.DataFrame(cities_weath)

In [36]:
cities_weather_df.forcast_time = pd.to_datetime(cities_weather_df.forcast_time)

In [37]:
cities_weather_df

,city_id,forcast_time,outlook,temperature,feels_like,wind_speed,rain_prob
0,1,2023-01-05 15:00:00,scattered clouds,11.89,10.99,2.55,0.00
1,1,2023-01-05 18:00:00,scattered clouds,11.27,10.44,2.60,0.00
2,1,2023-01-05 21:00:00,few clouds,10.60,9.83,4.29,0.00
3,1,2023-01-06 00:00:00,scattered clouds,10.21,9.51,5.69,0.00
4,1,2023-01-06 03:00:00,overcast clouds,10.02,9.35,5.32,0.00
...,...,...,...,...,...,...,...
319,36,2023-01-06 03:00:00,light rain,4.84,-0.32,8.94,0.99
320,36,2023-01-06 06:00:00,light rain,4.88,0.55,6.48,1.00
321,36,2023-01-06 09:00:00,light rain,5.50,1.26,6.72,0.84
322,36,2023-01-06 12:00:00,light rain,5.53,1.56,6.03,0.73


In [38]:
cities_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   city_id       324 non-null    int64         
 1   forcast_time  324 non-null    datetime64[ns]
 2   outlook       324 non-null    object        
 3   temperature   324 non-null    float64       
 4   feels_like    324 non-null    float64       
 5   wind_speed    324 non-null    float64       
 6   rain_prob     324 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 17.8+ KB


## Export Final df

In [24]:
cities_weather_df

,city_id,forcast_time,outlook,temperature,feels_like,wind_speed,rain_prob
0,1,2023-01-05 15:00:00,few clouds,11.89,10.99,2.55,0.00
1,1,2023-01-05 18:00:00,few clouds,11.27,10.44,2.60,0.00
2,1,2023-01-05 21:00:00,few clouds,10.60,9.83,4.29,0.00
3,1,2023-01-06 00:00:00,scattered clouds,10.21,9.51,5.69,0.00
4,1,2023-01-06 03:00:00,overcast clouds,10.02,9.35,5.32,0.00
...,...,...,...,...,...,...,...
319,36,2023-01-06 03:00:00,light rain,4.84,-0.32,8.94,0.99
320,36,2023-01-06 06:00:00,light rain,4.88,0.55,6.48,1.00
321,36,2023-01-06 09:00:00,light rain,5.50,1.26,6.72,0.84
322,36,2023-01-06 12:00:00,light rain,5.53,1.56,6.03,0.73


In [39]:
cities_weather_df.to_sql('weather',con=con,if_exists='append',index=False)

324